<a href="https://colab.research.google.com/github/DeepQn/datasets/blob/master/Level_Wise_Training_Saved_In_Drive.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install pytorch
!pip install torchvision==0.2.2

  Using cached https://files.pythonhosted.org/packages/ee/67/f403d4ae6e9cd74b546ee88cccdb29b8415a9c1b3d80aebeb20c9ea91d96/pytorch-1.0.2.tar.gz
  ERROR: Failed building wheel for pytorch
  Running setup.py clean for pytorch
Failed to build pytorch
  Running setup.py install for pytorch ... error
ERROR: Command "/usr/bin/python3 -u -c 'import setuptools, tokenize;__file__='"'"'/tmp/pip-install-eh4npw83/pytorch/setup.py'"'"';f=getattr(tokenize, '"'"'open'"'"', open)(__file__);code=f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' install --record /tmp/pip-record-o2d3ci42/install-record.txt --single-version-externally-managed --compile" failed with error code 1 in /tmp/pip-install-eh4npw83/pytorch/


In [3]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

E: Package 'python-software-properties' has no installation candidate
Selecting previously unselected package google-drive-ocamlfuse.
(Reading database ... 130811 files and directories currently installed.)
Preparing to unpack .../google-drive-ocamlfuse_0.7.3-0ubuntu3~ubuntu18.04.1_amd64.deb ...
Unpacking google-drive-ocamlfuse (0.7.3-0ubuntu3~ubuntu18.04.1) ...
Setting up google-drive-ocamlfuse (0.7.3-0ubuntu3~ubuntu18.04.1) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=ht

In [0]:
!mkdir -p drive
!google-drive-ocamlfuse drive

In [0]:
  !pkill -9 -f ipykernel_launcher

In [0]:
import torch
import torch.nn as nn
import torch.utils.data as data_utils
import torchvision
import torchvision.transforms as transforms
from torch.utils.data.sampler import SubsetRandomSampler
from torch.autograd import Variable
import time
import os.path
import numpy as np
import pickle

In [0]:
num_epochs = 40
batch_size = 500
learning_rate = 0.001
print_every = 20
best_accuracy = torch.FloatTensor([0])
start_epoch = 0
num_input_channel = 1
resume_weights = "drive/Deep/data/checkpoint_MNIST.pth.tar"

In [0]:
def init_var():
    global start_epoch, learning_rate, best_accuracy
    learning_rate = 0.001
    start_epoch = 0
    best_accuracy = torch.FloatTensor([0])

In [0]:
cuda = torch.cuda.is_available()
torch.manual_seed(1)
if cuda:
    torch.cuda.manual_seed(1)

transform = transforms.Compose([
    transforms.Resize((64, 64)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, ],
                         std=[0.5, ])
])


In [21]:
print("Loading the dataset...")
train_set = torchvision.datasets.MNIST(root='/input', train=True, download=True, transform=transform)
indices = list(range(len(train_set)))
val_split = 10000

val_idx = np.random.choice(indices, size=val_split, replace=False)
train_idx = list(set(indices) - set(val_idx))

val_sampler = SubsetRandomSampler(val_idx)
val_loader = torch.utils.data.DataLoader(dataset=train_set, batch_size=batch_size, sampler=val_sampler, shuffle=False)
val_loader2 = torch.utils.data.DataLoader(dataset=train_set, batch_size=1, sampler=val_sampler, shuffle=False)

train_sampler = SubsetRandomSampler(train_idx)
train_loader = torch.utils.data.DataLoader(dataset=train_set, batch_size=batch_size, sampler=train_sampler,
                                           shuffle=False)
train_loader2 = torch.utils.data.DataLoader(dataset=train_set, batch_size=1, sampler=train_sampler, shuffle=False)

test_set = torchvision.datasets.MNIST(root='/input', train=False, download=True, transform=transform)
test_loader = torch.utils.data.DataLoader(test_set, batch_size=batch_size, shuffle=False)
test_loader2 = torch.utils.data.DataLoader(test_set, batch_size=1, shuffle=False)

print("Dataset is Loaded")

print("Saving the dataset...")
pickle.dump(train_loader, open("drive/Deep/data/loaders/train_loader.txt", 'wb'))
pickle.dump(val_loader, open("drive/Deep/data/loaders/val_loader.txt", 'wb'))
pickle.dump(test_loader, open("drive/Deep/data/loaders/test_loader.txt", 'wb'))

pickle.dump(train_loader2, open("drive/Deep/data/loaders/train_loader2.txt", 'wb'))
pickle.dump(val_loader2, open("drive/Deep/data/loaders/val_loader2.txt", 'wb'))
pickle.dump(test_loader2, open("drive/Deep/data/loaders/test_loader2.txt", 'wb'))

Loading the dataset...
Dataset is Loaded
Saving the dataset...


In [23]:
print("Train Loader: ",len(train_loader))
print("Validation Loader: ",len(val_loader))
print("Test Loader: ",len(test_loader))

Train Loader:  100
Validation Loader:  20
Test Loader:  20


In [0]:
def train_cpu(model, optimizer, train_loader, loss_fun):
    average_time = 0
    total = 0
    correct = 0
    model.train()
    for i, (images, labels) in enumerate(train_loader):
        batch_time = time.time()
        images = Variable(images)
        labels = Variable(labels)

        if cuda:
            images, labels = images.cuda(), labels.cuda()

        optimizer.zero_grad()
        outputs = model(images)
        loss = loss_fun(outputs, labels)

        if cuda:
            loss.cpu()

        loss.backward()
        optimizer.step()

        batch_time = time.time() - batch_time
        average_time += batch_time

        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

        if (i + 1) % print_every == 0:
            print('Epoch: [%d/%d], Step: [%d/%d], Loss: %.4f, Accuracy: %.4f, Batch time: %f'
                  % (epoch + 1,
                     num_epochs,
                     i + 1,
                     len(train_loader),
                     loss.item(),
                     correct/total,
                     average_time / print_every))
            
           
def eval_cpu(model, test_loader):
    model.eval()

    total = 0
    correct = 0
    for i, (data, labels) in enumerate(test_loader):
        data, labels = Variable(data), Variable(labels)
        if cuda:
            data, labels = data.cuda(), labels.cuda()

        data = data.squeeze(0)
        labels = labels.squeeze(0)

        outputs = model(data)
        if cuda:
            outputs.cpu()

        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    return correct / total


def save_checkpoint(state, is_best, filename=resume_weights):
    if is_best:
        print("=> Saving a new best")
        torch.save(state, filename)
    else:
        print("=> Validation Accuracy did not improve")

In [0]:
def load_weights():
    pass

In [0]:
class Binary_Tree(nn.Module):
    def __init__(self):
        super(Binary_Tree, self).__init__()

        self.layer1 = nn.Sequential(
            nn.Conv2d(num_input_channel, 32, kernel_size=3, stride=2, padding=(2, 2)),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.BatchNorm2d(32),
            nn.ReLU())

        self.fc = nn.Sequential(
            nn.Linear(8192, 4096),
            nn.Dropout(0.5),
            nn.BatchNorm1d(4096),
            nn.ReLU())

        # final fc layer

        self.fc_final = nn.Sequential(
            nn.Linear(4096, 1024),
            nn.BatchNorm1d(1024),
            nn.ReLU())

    def forward(self, x):

        x1 = self.layer1(x)
        x1 = x1.view(-1, self.num_flat_features(x1))

        
        # final fc layer

        out = self.fc(x1)
        out = self.fc_final(out)
        
        return out

    def num_flat_features(self, x):
        size = x.size()[1:]
        num_features = 1
        for s in size:
            num_features *= s
        return num_features

In [0]:
def fit():
    init_var()
    global epoch, best_accuracy, learning_rate, start_epoch, model
    model = Binary_Tree()
    if cuda:
        model.cuda()
        
    load_weights()

    criterion = nn.CrossEntropyLoss()
    if cuda:
        criterion.cuda()

    total_step = len(train_loader)


    for epoch in range(num_epochs):

        print("\nLearning Rate:",learning_rate)

        optimizer = torch.optim.RMSprop(filter(lambda p: p.requires_grad, model.parameters()), lr=learning_rate)

        if learning_rate > 0.00003:
            learning_rate = learning_rate * 0.993

        train_cpu(model, optimizer, train_loader, criterion)
        val_acc = eval_cpu(model, val_loader)
        print('=> Validation set: Accuracy: {:.2f}%'.format(val_acc * 100))

        test_acc = eval_cpu(model, test_loader)
        print('=> Test set: Accuracy: {:.2f}%'.format(test_acc * 100))

        acc = torch.FloatTensor([val_acc])

        is_best = bool(acc.numpy() > best_accuracy.numpy())

        best_accuracy = torch.FloatTensor(max(acc.numpy(), best_accuracy.numpy()))

        save_checkpoint({
            'epoch': start_epoch + epoch + 1,
            'state_dict': model.state_dict(),
            'best_accuracy': best_accuracy
        }, is_best)

In [33]:
fit()


Learning Rate: 0.001
Epoch: [1/40], Step: [20/100], Loss: 5.4117, Accuracy: 0.4981, Batch time: 0.016995
Epoch: [1/40], Step: [40/100], Loss: 5.1272, Accuracy: 0.6625, Batch time: 0.031296
Epoch: [1/40], Step: [60/100], Loss: 4.9239, Accuracy: 0.7501, Batch time: 0.045997
Epoch: [1/40], Step: [80/100], Loss: 4.7767, Accuracy: 0.8010, Batch time: 0.060076
Epoch: [1/40], Step: [100/100], Loss: 4.6591, Accuracy: 0.8329, Batch time: 0.074618
=> Validation set: Accuracy: 89.66%
=> Test set: Accuracy: 89.81%
=> Saving a new best

Learning Rate: 0.000993
Epoch: [2/40], Step: [20/100], Loss: 4.3309, Accuracy: 0.9160, Batch time: 0.016633
Epoch: [2/40], Step: [40/100], Loss: 4.2192, Accuracy: 0.9406, Batch time: 0.030710
Epoch: [2/40], Step: [60/100], Loss: 4.0384, Accuracy: 0.9512, Batch time: 0.044799
Epoch: [2/40], Step: [80/100], Loss: 3.9369, Accuracy: 0.9570, Batch time: 0.059264
Epoch: [2/40], Step: [100/100], Loss: 3.8667, Accuracy: 0.9610, Batch time: 0.073627
=> Validation set: Accur

In [34]:
torch.cuda.memory_allocated(device=None)
torch.cuda.memory_cached(device=None)
torch.cuda.empty_cache()
torch.cuda.memory_cached(device=None)

765460480

In [0]:
layer = []
layer.append(model.layer1[0].weight)
layer.append(model.layer1[0].bias)

for (weight,i) in zip(layer,['w','b']*(len(layer)//2)):
    pickle.dump(weight, open("drive/Deep/data/weights/layer1"+i+".pkl", 'wb'))

In [0]:
class Binary_Tree(nn.Module):
    def __init__(self):
        super(Binary_Tree, self).__init__()

        self.layer1 = nn.Sequential(
            nn.Conv2d(num_input_channel, 32, kernel_size=3, stride=2, padding=(2, 2)),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.BatchNorm2d(32),
            nn.ReLU())
        
        self.layer21 = nn.Sequential(
            nn.Conv2d(32, 64, kernel_size=5, stride=1, padding=(2, 2)),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.BatchNorm2d(64),
            nn.ReLU())

        self.layer22 = nn.Sequential(
            nn.Conv2d(32, 64, kernel_size=5, stride=1, padding=(2, 2)),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.BatchNorm2d(64),
            nn.ReLU())

        self.fc = nn.Sequential(
            nn.Linear(8192, 4096),
            nn.Dropout(0.5),
            nn.BatchNorm1d(4096),
            nn.ReLU())

        # final fc layer

        self.fc_final = nn.Sequential(
            nn.Linear(4096, 1024),
            nn.BatchNorm1d(1024),
            nn.ReLU())

    def forward(self, x):

        x1 = self.layer1(x)
        x21 = self.layer21(x1)
        x22 = self.layer22(x1)

        x2 = torch.cat((x21, x22), 1)
        x2 = x2.view(-1, self.num_flat_features(x2))

        
        # final fc layer

        out = self.fc(x2)
        out = self.fc_final(out)
        
        return out

    def num_flat_features(self, x):
        size = x.size()[1:]
        num_features = 1
        for s in size:
            num_features *= s
        return num_features

In [0]:
def load_weights():
    net_layer1w = pickle.load(open("drive/Deep/data/weights/layer1w.pkl", 'rb'))
    net_layer1b = pickle.load(open("drive/Deep/data/weights/layer1b.pkl", 'rb'))

    model.layer1[0].weight = net_layer1w
    model.layer1[0].bias = net_layer1b

    model.layer1[0].weight.requires_grad = False
    model.layer1[0].bias.requires_grad = False

In [38]:
fit()


Learning Rate: 0.001
Epoch: [1/40], Step: [20/100], Loss: 5.1628, Accuracy: 0.6611, Batch time: 0.022573
Epoch: [1/40], Step: [40/100], Loss: 4.8959, Accuracy: 0.7994, Batch time: 0.041018
Epoch: [1/40], Step: [60/100], Loss: 4.7310, Accuracy: 0.8567, Batch time: 0.059602
Epoch: [1/40], Step: [80/100], Loss: 4.5987, Accuracy: 0.8864, Batch time: 0.079511
Epoch: [1/40], Step: [100/100], Loss: 4.4682, Accuracy: 0.9053, Batch time: 0.100740
=> Validation set: Accuracy: 97.96%
=> Test set: Accuracy: 98.22%
=> Validation Accuracy did not improve

Learning Rate: 0.000993
Epoch: [2/40], Step: [20/100], Loss: 4.2100, Accuracy: 0.9284, Batch time: 0.027779
Epoch: [2/40], Step: [40/100], Loss: 4.0268, Accuracy: 0.9552, Batch time: 0.048250
Epoch: [2/40], Step: [60/100], Loss: 3.8754, Accuracy: 0.9653, Batch time: 0.067001
Epoch: [2/40], Step: [80/100], Loss: 3.7689, Accuracy: 0.9707, Batch time: 0.085522
Epoch: [2/40], Step: [100/100], Loss: 3.6848, Accuracy: 0.9743, Batch time: 0.104129
=> Val

In [0]:
def load_weights():
    net_layer1w = pickle.load(open("drive/Deep/data/weights/layer1w.pkl", 'rb'))
    net_layer1b = pickle.load(open("drive/Deep/data/weights/layer1b.pkl", 'rb'))

    model.layer1[0].weight = net_layer1w
    model.layer1[0].bias = net_layer1b

    model.layer1[0].weight.requires_grad = False
    model.layer1[0].bias.requires_grad = False

In [0]:
layer = []
layer.append(model.layer21[0].weight)
layer.append(model.layer21[0].bias)
layer.append(model.layer22[0].weight)
layer.append(model.layer22[0].bias)


for (weight,i) in zip(layer,[str(x)+str(y) for x in [1,2] for y in ['w','b']]):
    pickle.dump(weight, open("drive/Deep/data/weights/layer2"+i+".pkl", 'wb'))

In [0]:
class Binary_Tree(nn.Module):
    def __init__(self):
        super(Binary_Tree, self).__init__()

        self.layer1 = nn.Sequential(
            nn.Conv2d(num_input_channel, 32, kernel_size=3, stride=2, padding=(2, 2)),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.BatchNorm2d(32),
            nn.ReLU())

        self.layer21 = nn.Sequential(
            nn.Conv2d(32, 64, kernel_size=5, stride=1, padding=(2, 2)),
            nn.BatchNorm2d(64),
            nn.ReLU())

        self.layer22 = nn.Sequential(
            nn.Conv2d(32, 64, kernel_size=5, stride=1, padding=(2, 2)),
            nn.BatchNorm2d(64),
            nn.ReLU())

        self.layer31 = nn.Sequential(
            nn.Conv2d(128, 256, kernel_size=7, stride=1, padding=(2, 2)),
            nn.MaxPool2d(kernel_size=4, stride=4),
            nn.BatchNorm2d(256),
            nn.ReLU())

        self.layer32 = nn.Sequential(
            nn.Conv2d(128, 256, kernel_size=7, stride=1, padding=(2, 2)),
            nn.MaxPool2d(kernel_size=4, stride=4),
            nn.BatchNorm2d(256),
            nn.ReLU())

        self.layer33 = nn.Sequential(
            nn.Conv2d(128, 256, kernel_size=7, stride=1, padding=(2, 2)),
            nn.MaxPool2d(kernel_size=4, stride=4),
            nn.BatchNorm2d(256),
            nn.ReLU())

        self.layer34 = nn.Sequential(
            nn.Conv2d(128, 256, kernel_size=7, stride=1, padding=(2, 2)),
            nn.MaxPool2d(kernel_size=4, stride=4),
            nn.BatchNorm2d(256),
            nn.ReLU())

        # final fc layer

        self.fc1 = nn.Sequential(
            nn.Linear(9216, 4096),
            nn.Dropout(0.5),
            nn.BatchNorm1d(4096),
            nn.ReLU())

        self.fc2 = nn.Sequential(
            nn.Linear(4096, 1024),
            nn.BatchNorm1d(1024),
            nn.ReLU())

    def forward(self, x):
        x1 = self.layer1(x)

        x21 = self.layer21(x1)

        x22 = self.layer22(x1)

        x2 = torch.cat((x21, x22), 1)

        x31 = self.layer31(x2)

        x32 = self.layer32(x2)

        x33 = self.layer33(x2)

        x34 = self.layer34(x2)

        x3 = torch.cat((x31, x32), 1)
        x3 = torch.cat((x3, x33), 1)
        x3 = torch.cat((x3, x34), 1)

        x3 = x3.view(-1, self.num_flat_features(x3))

        # final fc layer

        out = self.fc1(x3)
        out = self.fc2(out)

        return out

    def num_flat_features(self, x):
        size = x.size()[1:]
        num_features = 1
        for s in size:
            num_features *= s
        return num_features

In [0]:
model = Binary_Tree()
if cuda:
    model.cuda()

def load_weights():
    net_layer1w = pickle.load(open("drive/Deep/data/weights/layer1w.pkl", 'rb'))
    net_layer1b = pickle.load(open("drive/Deep/data/weights/layer1b.pkl", 'rb'))

    model.layer1[0].weight = net_layer1w
    model.layer1[0].bias = net_layer1b

    model.layer1[0].weight.requires_grad = False
    model.layer1[0].bias.requires_grad = False
    
    net_layer21w = pickle.load(open("drive/Deep/data/weights/layer21w.pkl", 'rb'))
    net_layer21b = pickle.load(open("drive/Deep/data/weights/layer21b.pkl", 'rb'))
    
    net_layer22w = pickle.load(open("drive/Deep/data/weights/layer22w.pkl", 'rb'))
    net_layer22b = pickle.load(open("drive/Deep/data/weights/layer22b.pkl", 'rb'))


    model.layer21[0].weight = net_layer21w
    model.layer21[0].bias = net_layer21b
    model.layer22[0].weight = net_layer22w
    model.layer22[0].bias = net_layer22b

    model.layer21[0].weight.requires_grad = False
    model.layer21[0].bias.requires_grad = False
    model.layer22[0].weight.requires_grad = False
    model.layer22[0].bias.requires_grad = False

In [68]:
torch.cuda.memory_allocated(device=None)
torch.cuda.memory_cached(device=None)
torch.cuda.empty_cache()
torch.cuda.memory_cached(device=None)

3554672640

In [67]:
fit()


Learning Rate: 0.001
Epoch: [1/40], Step: [20/100], Loss: 5.5172, Accuracy: 0.5441, Batch time: 0.262333
Epoch: [1/40], Step: [40/100], Loss: 5.2623, Accuracy: 0.7416, Batch time: 0.521526
Epoch: [1/40], Step: [60/100], Loss: 5.1035, Accuracy: 0.8203, Batch time: 0.783685
Epoch: [1/40], Step: [80/100], Loss: 4.9487, Accuracy: 0.8617, Batch time: 1.046936
Epoch: [1/40], Step: [100/100], Loss: 4.8910, Accuracy: 0.8866, Batch time: 1.312414
=> Validation set: Accuracy: 98.77%
=> Test set: Accuracy: 98.87%
=> Validation Accuracy did not improve

Learning Rate: 0.000993
Epoch: [2/40], Step: [20/100], Loss: 4.6112, Accuracy: 0.9255, Batch time: 0.269378
Epoch: [2/40], Step: [40/100], Loss: 4.4428, Accuracy: 0.9564, Batch time: 0.540416
Epoch: [2/40], Step: [60/100], Loss: 4.2892, Accuracy: 0.9669, Batch time: 0.813359
Epoch: [2/40], Step: [80/100], Loss: 4.1388, Accuracy: 0.9732, Batch time: 1.087277
Epoch: [2/40], Step: [100/100], Loss: 4.1008, Accuracy: 0.9768, Batch time: 1.363188
=> Val

KeyboardInterrupt: ignored

In [69]:
print(torch.cuda.memory_allocated(device=None))
print(torch.cuda.memory_cached(device=None))
torch.cuda.empty_cache()

2121748480
3554672640


In [0]:
layer = []
layer.append(model.layer31[0].weight)
layer.append(model.layer31[0].bias)
layer.append(model.layer32[0].weight)
layer.append(model.layer32[0].bias)
layer.append(model.layer33[0].weight)
layer.append(model.layer33[0].bias)
layer.append(model.layer34[0].weight)
layer.append(model.layer34[0].bias)


for (weight,i) in zip(layer,[str(x)+str(y) for x in [1,2,3,4] for y in ['w','b']]):
    pickle.dump(weight, open("drive/Deep/data/weights/layer3"+i+".pkl", 'wb'))

In [0]:
class Binary_Tree(nn.Module):
    def __init__(self):
        super(Binary_Tree, self).__init__()

        self.layer1 = nn.Sequential(
            nn.Conv2d(num_input_channel, 32, kernel_size=3, stride=2, padding=(2, 2)),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.BatchNorm2d(32),
            nn.ReLU())

        self.layer21 = nn.Sequential(
            nn.Conv2d(32, 64, kernel_size=5, stride=1, padding=(2, 2)),
            nn.BatchNorm2d(64),
            nn.ReLU())

        self.layer22 = nn.Sequential(
            nn.Conv2d(32, 64, kernel_size=5, stride=1, padding=(2, 2)),
            nn.BatchNorm2d(64),
            nn.ReLU())

        self.layer31 = nn.Sequential(
            nn.Conv2d(128, 256, kernel_size=7, stride=1, padding=(2, 2)),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.BatchNorm2d(256),
            nn.ReLU())

        self.layer32 = nn.Sequential(
            nn.Conv2d(128, 256, kernel_size=7, stride=1, padding=(2, 2)),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.BatchNorm2d(256),
            nn.ReLU())

        self.layer33 = nn.Sequential(
            nn.Conv2d(128, 256, kernel_size=7, stride=1, padding=(2, 2)),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.BatchNorm2d(256),
            nn.ReLU())

        self.layer34 = nn.Sequential(
            nn.Conv2d(128, 256, kernel_size=7, stride=1, padding=(2, 2)),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.BatchNorm2d(256),
            nn.ReLU())

        self.layer41 = nn.Sequential(
            nn.Conv2d(1024, 1024, kernel_size=3, stride=2, padding=(2, 2)),
            nn.MaxPool2d(kernel_size=4, stride=2),
            nn.BatchNorm2d(1024),
            nn.ReLU())

        self.layer42 = nn.Sequential(
            nn.Conv2d(1024, 1024, kernel_size=3, stride=2, padding=(2, 2)),
            nn.MaxPool2d(kernel_size=4, stride=2),
            nn.BatchNorm2d(1024),
            nn.ReLU())

        self.layer43 = nn.Sequential(
            nn.Conv2d(1024, 1024, kernel_size=3, stride=2, padding=(2, 2)),
            nn.MaxPool2d(kernel_size=4, stride=2),
            nn.BatchNorm2d(1024),
            nn.ReLU())

        self.layer44 = nn.Sequential(
            nn.Conv2d(1024, 1024, kernel_size=3, stride=2, padding=(2, 2)),
            nn.MaxPool2d(kernel_size=4, stride=2),
            nn.BatchNorm2d(1024),
            nn.ReLU())

        self.layer45 = nn.Sequential(
            nn.Conv2d(1024, 1024, kernel_size=3, stride=2, padding=(2, 2)),
            nn.MaxPool2d(kernel_size=4, stride=2),
            nn.BatchNorm2d(1024),
            nn.ReLU())

        self.layer46 = nn.Sequential(
            nn.Conv2d(1024, 1024, kernel_size=3, stride=2, padding=(2, 2)),
            nn.MaxPool2d(kernel_size=4, stride=2),
            nn.BatchNorm2d(1024),
            nn.ReLU())

        self.layer47 = nn.Sequential(
            nn.Conv2d(1024, 1024, kernel_size=3, stride=2, padding=(2, 2)),
            nn.MaxPool2d(kernel_size=4, stride=2),
            nn.BatchNorm2d(1024),
            nn.ReLU())

        self.layer48 = nn.Sequential(
            nn.Conv2d(1024, 1024, kernel_size=3, stride=2, padding=(2, 2)),
            nn.MaxPool2d(kernel_size=4, stride=2),
            nn.BatchNorm2d(1024),
            nn.ReLU())

        # final fc layer

        self.fc1 = nn.Sequential(
            nn.Linear(8192, 4096),
            nn.Dropout(0.5),
            nn.BatchNorm1d(4096),
            nn.ReLU())

        self.fc2 = nn.Sequential(
            nn.Linear(4096, 1024),
            nn.BatchNorm1d(1024),
            nn.ReLU())

    def forward(self, x):
        x1 = self.layer1(x)

        x21 = self.layer21(x1)

        x22 = self.layer22(x1)

        x2 = torch.cat((x21, x22), 1)

        x31 = self.layer31(x2)

        x32 = self.layer32(x2)

        x33 = self.layer33(x2)

        x34 = self.layer34(x2)

        x3 = torch.cat((x31, x32), 1)
        x3 = torch.cat((x3, x33), 1)
        x3 = torch.cat((x3, x34), 1)

        x41 = self.layer41(x3)

        x42 = self.layer42(x3)

        x43 = self.layer43(x3)

        x44 = self.layer44(x3)

        x45 = self.layer45(x3)

        x46 = self.layer46(x3)

        x47 = self.layer47(x3)

        x48 = self.layer48(x3)

        x4 = torch.cat((x41, x42), 1)
        x4 = torch.cat((x4, x43), 1)
        x4 = torch.cat((x4, x44), 1)
        x4 = torch.cat((x4, x45), 1)
        x4 = torch.cat((x4, x46), 1)
        x4 = torch.cat((x4, x47), 1)
        x4 = torch.cat((x4, x48), 1)

        x4 = x4.view(-1, self.num_flat_features(x4))

        # final fc layer

        out = self.fc1(x4)
        out = self.fc2(out)

        return out

    def num_flat_features(self, x):
        size = x.size()[1:]
        num_features = 1
        for s in size:
            num_features *= s
        return num_features

In [0]:
def load_weights():
    net_layer1w = pickle.load(open("drive/Deep/data/weights/layer1w.pkl", 'rb'))
    net_layer1b = pickle.load(open("drive/Deep/data/weights/layer1b.pkl", 'rb'))

    model.layer1[0].weight = net_layer1w
    model.layer1[0].bias = net_layer1b

    model.layer1[0].weight.requires_grad = False
    model.layer1[0].bias.requires_grad = False
    
    net_layer21w = pickle.load(open("drive/Deep/data/weights/layer21w.pkl", 'rb'))
    net_layer21b = pickle.load(open("drive/Deep/data/weights/layer21b.pkl", 'rb'))
    
    net_layer22w = pickle.load(open("drive/Deep/data/weights/layer22w.pkl", 'rb'))
    net_layer22b = pickle.load(open("drive/Deep/data/weights/layer22b.pkl", 'rb'))


    model.layer21[0].weight = net_layer21w
    model.layer21[0].bias = net_layer21b
    model.layer22[0].weight = net_layer22w
    model.layer22[0].bias = net_layer22b

    model.layer21[0].weight.requires_grad = False
    model.layer21[0].bias.requires_grad = False
    model.layer22[0].weight.requires_grad = False
    model.layer22[0].bias.requires_grad = False
    
    net_layer31w = pickle.load(open("drive/Deep/data/weights/layer31w.pkl", 'rb'))
    net_layer31b = pickle.load(open("drive/Deep/data/weights/layer31b.pkl", 'rb'))
    
    net_layer32w = pickle.load(open("drive/Deep/data/weights/layer32w.pkl", 'rb'))
    net_layer32b = pickle.load(open("drive/Deep/data/weights/layer32b.pkl", 'rb'))
    
    net_layer33w = pickle.load(open("drive/Deep/data/weights/layer33w.pkl", 'rb'))
    net_layer33b = pickle.load(open("drive/Deep/data/weights/layer33b.pkl", 'rb'))
    
    net_layer34w = pickle.load(open("drive/Deep/data/weights/layer34w.pkl", 'rb'))
    net_layer34b = pickle.load(open("drive/Deep/data/weights/layer34b.pkl", 'rb'))
    
    model.layer31[0].weight = net_layer31w
    model.layer31[0].bias = net_layer31b
    model.layer32[0].weight = net_layer32w
    model.layer32[0].bias = net_layer32b
    model.layer33[0].weight = net_layer33w
    model.layer33[0].bias = net_layer33b
    model.layer34[0].weight = net_layer34w
    model.layer34[0].bias = net_layer34b

    model.layer31[0].weight.requires_grad = False
    model.layer31[0].bias.requires_grad = False
    model.layer32[0].weight.requires_grad = False
    model.layer32[0].bias.requires_grad = False
    model.layer33[0].weight.requires_grad = False
    model.layer33[0].bias.requires_grad = False
    model.layer34[0].weight.requires_grad = False
    model.layer34[0].bias.requires_grad = False


In [81]:
fit()


Learning Rate: 0.001
Epoch: [1/40], Step: [20/100], Loss: 4.9289, Accuracy: 0.7215, Batch time: 0.665677
Epoch: [1/40], Step: [40/100], Loss: 4.7037, Accuracy: 0.8331, Batch time: 1.357757
Epoch: [1/40], Step: [60/100], Loss: 4.5611, Accuracy: 0.8848, Batch time: 2.055426
Epoch: [1/40], Step: [80/100], Loss: 4.4074, Accuracy: 0.9114, Batch time: 2.751813
Epoch: [1/40], Step: [100/100], Loss: 4.2831, Accuracy: 0.9274, Batch time: 3.449901
=> Validation set: Accuracy: 98.83%
=> Test set: Accuracy: 99.00%
=> Validation Accuracy did not improve

Learning Rate: 0.000993
Epoch: [2/40], Step: [20/100], Loss: 4.0319, Accuracy: 0.9296, Batch time: 0.696196
Epoch: [2/40], Step: [40/100], Loss: 3.8369, Accuracy: 0.9613, Batch time: 1.393010
Epoch: [2/40], Step: [60/100], Loss: 3.7169, Accuracy: 0.9722, Batch time: 2.089178
Epoch: [2/40], Step: [80/100], Loss: 3.5795, Accuracy: 0.9784, Batch time: 2.784602
Epoch: [2/40], Step: [100/100], Loss: 3.4655, Accuracy: 0.9821, Batch time: 3.480437
=> Val

KeyboardInterrupt: ignored

In [82]:
print(torch.cuda.memory_allocated(device=None))
print(torch.cuda.memory_cached(device=None))
torch.cuda.empty_cache()

1988892160
7788822528


In [0]:
layer = []
layer.append(model.layer41[0].weight)
layer.append(model.layer41[0].bias)
layer.append(model.layer42[0].weight)
layer.append(model.layer42[0].bias)
layer.append(model.layer43[0].weight)
layer.append(model.layer43[0].bias)
layer.append(model.layer44[0].weight)
layer.append(model.layer44[0].bias)
layer.append(model.layer45[0].weight)
layer.append(model.layer45[0].bias)
layer.append(model.layer46[0].weight)
layer.append(model.layer46[0].bias)
layer.append(model.layer47[0].weight)
layer.append(model.layer47[0].bias)
layer.append(model.layer48[0].weight)
layer.append(model.layer48[0].bias)


for (weight,i) in zip(layer,[str(x)+str(y) for x in [1,2,3,4,5,6,7,8] for y in ['w','b']]):
    pickle.dump(weight, open("drive/Deep/data/weights/layer4"+i+".pkl", 'wb'))

In [0]:
def load_weights():
    net_layer1w = pickle.load(open("drive/Deep/data/weights/layer1w.pkl", 'rb'))
    net_layer1b = pickle.load(open("drive/Deep/data/weights/layer1b.pkl", 'rb'))

    model.layer1[0].weight = net_layer1w
    model.layer1[0].bias = net_layer1b

    net_layer21w = pickle.load(open("drive/Deep/data/weights/layer21w.pkl", 'rb'))
    net_layer21b = pickle.load(open("drive/Deep/data/weights/layer21b.pkl", 'rb'))
    
    net_layer22w = pickle.load(open("drive/Deep/data/weights/layer22w.pkl", 'rb'))
    net_layer22b = pickle.load(open("drive/Deep/data/weights/layer22b.pkl", 'rb'))


    model.layer21[0].weight = net_layer21w
    model.layer21[0].bias = net_layer21b
    model.layer22[0].weight = net_layer22w
    model.layer22[0].bias = net_layer22b
    
    net_layer31w = pickle.load(open("drive/Deep/data/weights/layer31w.pkl", 'rb'))
    net_layer31b = pickle.load(open("drive/Deep/data/weights/layer31b.pkl", 'rb'))
    
    net_layer32w = pickle.load(open("drive/Deep/data/weights/layer32w.pkl", 'rb'))
    net_layer32b = pickle.load(open("drive/Deep/data/weights/layer32b.pkl", 'rb'))
    
    net_layer33w = pickle.load(open("drive/Deep/data/weights/layer33w.pkl", 'rb'))
    net_layer33b = pickle.load(open("drive/Deep/data/weights/layer33b.pkl", 'rb'))
    
    net_layer34w = pickle.load(open("drive/Deep/data/weights/layer34w.pkl", 'rb'))
    net_layer34b = pickle.load(open("drive/Deep/data/weights/layer34b.pkl", 'rb'))


    model.layer31[0].weight = net_layer31w
    model.layer31[0].bias = net_layer31b
    model.layer32[0].weight = net_layer32w
    model.layer32[0].bias = net_layer32b
    model.layer33[0].weight = net_layer33w
    model.layer33[0].bias = net_layer33b
    model.layer34[0].weight = net_layer34w
    model.layer34[0].bias = net_layer34b
    
    net_layer41w = pickle.load(open("drive/Deep/data/weights/layer41w.pkl", 'rb'))
    net_layer41b = pickle.load(open("drive/Deep/data/weights/layer41b.pkl", 'rb'))
    
    net_layer42w = pickle.load(open("drive/Deep/data/weights/layer42w.pkl", 'rb'))
    net_layer42b = pickle.load(open("drive/Deep/data/weights/layer42b.pkl", 'rb'))
    
    net_layer43w = pickle.load(open("drive/Deep/data/weights/layer43w.pkl", 'rb'))
    net_layer43b = pickle.load(open("drive/Deep/data/weights/layer43b.pkl", 'rb'))
    
    net_layer44w = pickle.load(open("drive/Deep/data/weights/layer44w.pkl", 'rb'))
    net_layer44b = pickle.load(open("drive/Deep/data/weights/layer44b.pkl", 'rb'))
    
    net_layer45w = pickle.load(open("drive/Deep/data/weights/layer45w.pkl", 'rb'))
    net_layer45b = pickle.load(open("drive/Deep/data/weights/layer45b.pkl", 'rb'))
    
    net_layer46w = pickle.load(open("drive/Deep/data/weights/layer46w.pkl", 'rb'))
    net_layer46b = pickle.load(open("drive/Deep/data/weights/layer46b.pkl", 'rb'))
    
    net_layer47w = pickle.load(open("drive/Deep/data/weights/layer47w.pkl", 'rb'))
    net_layer47b = pickle.load(open("drive/Deep/data/weights/layer47b.pkl", 'rb'))
    
    net_layer48w = pickle.load(open("drive/Deep/data/weights/layer48w.pkl", 'rb'))
    net_layer48b = pickle.load(open("drive/Deep/data/weights/layer48b.pkl", 'rb'))
    
    model.layer41[0].weight = net_layer41w
    model.layer41[0].bias = net_layer41b
    model.layer42[0].weight = net_layer42w
    model.layer42[0].bias = net_layer42b
    model.layer43[0].weight = net_layer43w
    model.layer43[0].bias = net_layer43b
    model.layer44[0].weight = net_layer44w
    model.layer44[0].bias = net_layer44b
    model.layer45[0].weight = net_layer45w
    model.layer45[0].bias = net_layer45b
    model.layer46[0].weight = net_layer46w
    model.layer46[0].bias = net_layer46b
    model.layer47[0].weight = net_layer47w
    model.layer47[0].bias = net_layer47b
    model.layer48[0].weight = net_layer48w
    model.layer48[0].bias = net_layer48b


In [0]:
fit()


Learning Rate: 0.001
Epoch: [1/40], Step: [20/100], Loss: 4.7954, Accuracy: 0.7884, Batch time: 0.671073
Epoch: [1/40], Step: [40/100], Loss: 4.6037, Accuracy: 0.8890, Batch time: 1.365980
Epoch: [1/40], Step: [60/100], Loss: 4.4544, Accuracy: 0.9256, Batch time: 2.066078
Epoch: [1/40], Step: [80/100], Loss: 4.3377, Accuracy: 0.9439, Batch time: 2.766815
Epoch: [1/40], Step: [100/100], Loss: 4.2193, Accuracy: 0.9550, Batch time: 3.466339
=> Validation set: Accuracy: 99.31%
=> Test set: Accuracy: 99.43%
=> Saving a new best

Learning Rate: 0.000993
Epoch: [2/40], Step: [20/100], Loss: 3.9075, Accuracy: 0.9749, Batch time: 0.703226
Epoch: [2/40], Step: [40/100], Loss: 3.7519, Accuracy: 0.9872, Batch time: 1.403156
Epoch: [2/40], Step: [60/100], Loss: 3.6280, Accuracy: 0.9914, Batch time: 2.102942
Epoch: [2/40], Step: [80/100], Loss: 3.4881, Accuracy: 0.9935, Batch time: 2.801720
Epoch: [2/40], Step: [100/100], Loss: 3.4083, Accuracy: 0.9947, Batch time: 3.500047
=> Validation set: Accur